# Import related functions

In [1]:
import cobra
#from script.ECMpy_function import *
import sys
sys.path.append(r'./script/')
from ECMpy_function import *

# Input and output files

In [2]:
ecModel_file="./model/eciML1515.json"
obj='EX_trp__L_e' # EX_lys_L_e
substrate = 'EX_glc__D_e'
substrate_con=10
biomass_id = 'BIOMASS_Ec_iML1515_core_75p37M'
biomass_min=0.1

fluxes_outfile = './analysis/ECMpy_solution_%s_%.2g_pfba.csv'%(obj,substrate_con)
enzcost_cost_file = './analysis/enzcost_%s_%.2g.csv'%(obj,substrate_con)
enzcost_diff_file = './analysis/enzcost_diff_%s_%.2g.csv'%(obj,substrate_con)
enzcost_diff_selcet_file = './analysis/enzcost_diff_%s_%.2g_select.csv'%(obj,substrate_con)
FSEOF_file = './analysis/FESEOF_%s_%.2g.csv'%(obj,substrate_con)

# Directly determining targets based on enzyme abundance

In [3]:
enzcost = pd.DataFrame()

enz_model=get_enzyme_constraint_model(ecModel_file)
enz_model.reactions.get_by_id(substrate).bounds=(-substrate_con,0)#EX_glc_e EX_fru_e EX_sucr_e EX_inost_e EX_lac_D_e EX_ac_e
enz_model.reactions.get_by_id('%s_reverse'%substrate).bounds=(0,0)
enz_model.reactions.get_by_id(biomass_id).bounds=(biomass_min,biomass_min)
enz_model.objective=obj
enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)

enzcost = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,ecModel_file)
enzcost['enz_ratio'] = enzcost['E']/np.sum(enzcost['E'])
enzcost = enzcost.sort_values('enz_ratio', ascending=False)
enzcost_select = enzcost[enzcost['enz_ratio']>0.01]
enzcost_select.to_csv(enzcost_cost_file)
enzcost_select.head(10)

,fluxes,kcat_MW,E,equ,ec-code,enz_ratio
ANS,3.594316,76.627794,0.046906,chor_c + gln__L_c --> anth_c + glu__L_c + h_c ...,4.1.3.27,0.206635
TRPAS2_reverse,3.594316,85.270150,0.042152,indole_c + nh4_c + pyr_c --> h2o_c + trp__L_c,NaN,0.185692
ANPRT,3.594316,158.255810,0.022712,anth_c + prpp_c --> ppi_c + pran_c,2.4.2.18,0.100053
TRPS3,3.594316,249.207417,0.014423,3ig3p_c --> g3p_c + indole_c,"4.1.2.8,4.2.1.20",0.063537
PRAIi,3.594316,363.692938,0.009883,pran_c --> 2cpr5p_c,5.3.1.24,0.043537
IGPS,3.594316,363.692938,0.009883,2cpr5p_c + h_c --> 3ig3p_c + co2_c + h2o_c,4.1.1.48,0.043537
GLCDpp_num2,5.074624,1061.316075,0.004781,glc__D_p + h2o_p + q8_c --> glcn_p + h_p + q8h2_c,NaN,0.021064
PRPPS,3.706289,841.654904,0.004404,atp_c + r5p_c --> amp_c + h_c + prpp_c,2.7.6.1,0.019399
GNK_num2,5.074624,1273.803729,0.003984,atp_c + glcn_c --> 6pgc_c + adp_c + h_c,2.7.1.12,0.017550
PDH,2.960121,784.108302,0.003775,coa_c + nad_c + pyr_c --> accoa_c + co2_c + na...,"1.2.1.-,1.2.1.51,1.2.4.1,1.8.1.4,2.3.1.12",0.016631


#  Determine targets based on the fold changes of enzyme cost 

In [4]:
biomass_max=0.6
FC_threshold = 1.5

enzcost_diff = get_enz_foldchange(ecModel_file,obj,substrate,substrate_con,biomass_id,biomass_min,biomass_max,FC_threshold,fluxes_outfile, enzcost_diff_file)
enzcost_diff_select = enzcost_diff[enzcost_diff['E_μ0.1']>0.0001]
enzcost_diff_select.to_csv(enzcost_diff_selcet_file)

enzcost_diff_select.head(10)

/home/maozt/anaconda3/envs/ECMpy2/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


,E_μ0.1,E_μ0.6,log2_foldchange(max/min),log2_foldchange(min/max),type,foldchange(max/min),equ,ec-code
3OAS160_num1,0.000117,0.000702,2.584963,-2.584963,weaken_target,6.0000,h_c + malACP_c + myrsACP_c --> 3opalmACP_c + A...,2.3.1.86
3OAR160,0.000140,0.000842,2.584963,-2.584963,weaken_target,6.0000,3opalmACP_c + h_c + nadph_c --> 3hpalmACP_c + ...,"1.1.1.100,2.3.1.-,2.3.1.85,2.3.1.86"
3OAR140,0.000449,0.002693,2.584963,-2.584963,weaken_target,6.0000,3omrsACP_c + h_c + nadph_c --> 3hmrsACP_c + na...,"1.1.1.100,2.3.1.-,2.3.1.85,2.3.1.86"
3OAS140_num1,0.000115,0.000690,2.584963,-2.584963,weaken_target,6.0000,ddcaACP_c + h_c + malACP_c --> 3omrsACP_c + AC...,2.3.1.86
ACONTa_num1,0.000580,0.002436,2.070767,-2.070767,weaken_target,4.2011,cit_c --> acon_C_c + h2o_c,4.2.1.3
EAR121x,0.000119,0.000714,2.584963,-2.584963,weaken_target,6.0000,h_c + nadh_c + t3c5ddeceACP_c --> cddec5eACP_c...,1.3.1.9
3OAR161,0.000165,0.000992,2.584963,-2.584963,weaken_target,6.0000,3ocpalm9eACP_c + h_c + nadph_c --> 3hcpalm9eAC...,1.1.1.100
3OAR141,0.000165,0.000992,2.584963,-2.584963,weaken_target,6.0000,3ocmrs7eACP_c + h_c + nadph_c --> 3hcmrs7eACP_...,1.1.1.100
EAR160y,0.000153,0.000918,2.584963,-2.584963,weaken_target,6.0000,h_c + nadph_c + tpalm2eACP_c --> nadp_c + palm...,1.3.1.10
EAR161y,0.000179,0.001072,2.584963,-2.584963,weaken_target,6.0000,h_c + nadph_c + t3c9palmeACP_c --> hdeACP_c + ...,1.3.1.10


# Determine targets based on FESOF

In [5]:
model=get_enzyme_constraint_model(ecModel_file)

FSEOFdf_done = run_FSEOF(model,substrate,substrate_con,biomass_id,obj,FSEOF_file)

FSEOFdf_done.head(10)

./script/ECMpy_function.py:2653: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  always_down['regulation'] = 'down'
./script/ECMpy_function.py:2656: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  always_up['regulation'] = 'up'


,growth = 0.34,FC growth = 0.34,growth = 0.37,FC growth = 0.37,growth = 0.4,FC growth = 0.4,growth = 0.43,FC growth = 0.43,growth = 0.46,FC growth = 0.46,...,FC growth = 0.55,growth = 0.58,FC growth = 0.58,growth = 0.61,FC growth = 0.61,WT,GPR,FC_mean,regulation,reactions
id,,,,,,,,,,,,,,,,,,,,,
H2Otpp_reverse_num2,39.134690,1000.000000,38.784008,1000.000000,38.433325,1000.000000,38.082642,1000.000000,37.731959,1000.000000,...,1000.000000,36.083873,1000.000000,35.649234,1000.000000,0.000000,b0875,562.598358,up,h2o_c --> h2o_p
H2Otex_reverse_num8,39.134690,1000.000000,38.784008,1000.000000,38.433325,1000.000000,38.082642,1000.000000,37.731959,1000.000000,...,1000.000000,36.083873,1000.000000,35.649234,1000.000000,0.000000,b0929,562.598358,up,h2o_p --> h2o_e
CO2tex_reverse_num4,13.119677,1000.000000,13.437758,1000.000000,13.755840,1000.000000,14.073921,1000.000000,14.392003,1000.000000,...,1000.000000,15.630991,1000.000000,15.937058,1000.000000,0.000000,b2215,551.995516,up,co2_p --> co2_e
Htex_reverse_num4,12.132625,1000.000000,12.272701,1000.000000,12.412777,1000.000000,12.552853,1000.000000,12.692929,1000.000000,...,1000.000000,13.362659,1000.000000,13.540216,1000.000000,0.000000,b2215,551.227208,up,h_p --> h_e
ACtex_reverse_num4,4.749431,1000.000000,4.979100,1000.000000,5.208770,1000.000000,5.438439,1000.000000,5.668108,1000.000000,...,1000.000000,6.749216,1000.000000,7.034352,1000.000000,0.000000,b2215,548.035665,up,ac_p --> ac_e
TRPtex_reverse_num4,2.129569,1000.000000,1.946947,1000.000000,1.764324,1000.000000,1.581701,1000.000000,1.399079,1000.000000,...,1000.000000,0.642086,1000.000000,0.450470,1000.000000,0.000000,b2215,546.089712,up,trp__L_p --> trp__L_e
TRPt2rpp_reverse_num1,2.129569,1000.000000,1.946947,1000.000000,1.764324,1000.000000,1.581701,1000.000000,1.399079,1000.000000,...,1000.000000,0.642086,1000.000000,0.450470,1000.000000,0.000000,b3709,546.089712,up,h_c + trp__L_c --> h_p + trp__L_p
ANS,2.148896,55.574245,1.967978,50.895402,1.787061,46.216558,1.606144,41.537715,1.425226,36.858872,...,22.369518,0.675055,17.458107,0.485145,12.546695,0.038667,b1263 and b1264,22.007804,up,chor_c + gln__L_c --> anth_c + glu__L_c + h_c ...
TRPS3,2.148896,55.574245,1.967978,50.895402,1.787061,46.216558,1.606144,41.537715,1.425226,36.858872,...,22.369518,0.675055,17.458107,0.485145,12.546695,0.038667,b1260 and b1261,22.007804,up,3ig3p_c --> g3p_c + indole_c
